In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2


In [2]:
# Read in the files as Pandas DataFrames.
postsecdata_to_load = "postsecondary_enrollment_combined_1718_1819_1920.csv"
post_sec_df = pd.read_csv(postsecdata_to_load)


In [3]:
post_sec_df.head(10)

,SCHOOL_YEAR,AGENCY_TYPE,CHARTER_IND,CESA,COUNTY,DISTRICT_ID,SCHOOL_CODE,GRADE_GROUP,DISTRICT_NAME,SCHOOL_NAME,INITIAL_ENROLLMENT,INSTITUTION_LOCATION,INSTITUTION_LEVEL,INSTITUTION_TYPE,GROUP_BY,GROUP_BY_VALUE,GROUP_COUNT,STUDENT_COUNT
0,2019-20,NaN,NaN,NaN,NaN,0,NaN,[All],[Statewide],[Statewide],First Fall,In-State,2-Year,Public,All Students,All Students,32744,7097
1,2019-20,NaN,NaN,NaN,NaN,0,NaN,[All],[Statewide],[Statewide],First Fall,In-State,4-Year,Multiple,All Students,All Students,32744,211
2,2019-20,NaN,NaN,NaN,NaN,0,NaN,[All],[Statewide],[Statewide],First Fall,In-State,4-Year,Private,All Students,All Students,32744,2171
3,2019-20,NaN,NaN,NaN,NaN,0,NaN,[All],[Statewide],[Statewide],First Fall,In-State,4-Year,Public,All Students,All Students,32744,11695
4,2019-20,NaN,NaN,NaN,NaN,0,NaN,[All],[Statewide],[Statewide],First Fall,In-State,Multiple,Multiple,All Students,All Students,32744,314
5,2019-20,NaN,NaN,NaN,NaN,0,NaN,[All],[Statewide],[Statewide],First Fall,In-State,Multiple,Public,All Students,All Students,32744,1169
6,2019-20,NaN,NaN,NaN,NaN,0,NaN,[All],[Statewide],[Statewide],First Fall,Multiple,2-Year,Multiple,All Students,All Students,32744,2
7,2019-20,NaN,NaN,NaN,NaN,0,NaN,[All],[Statewide],[Statewide],First Fall,Multiple,2-Year,Public,All Students,All Students,32744,65
8,2019-20,NaN,NaN,NaN,NaN,0,NaN,[All],[Statewide],[Statewide],First Fall,Multiple,4-Year,Multiple,All Students,All Students,32744,208
9,2019-20,NaN,NaN,NaN,NaN,0,NaN,[All],[Statewide],[Statewide],First Fall,Multiple,4-Year,Private,All Students,All Students,32744,66


In [4]:
post_sec_df.dtypes

SCHOOL_YEAR              object
AGENCY_TYPE              object
CHARTER_IND              object
CESA                    float64
COUNTY                   object
DISTRICT_ID               int64
SCHOOL_CODE             float64
GRADE_GROUP              object
DISTRICT_NAME            object
SCHOOL_NAME              object
INITIAL_ENROLLMENT       object
INSTITUTION_LOCATION     object
INSTITUTION_LEVEL        object
INSTITUTION_TYPE         object
GROUP_BY                 object
GROUP_BY_VALUE           object
GROUP_COUNT              object
STUDENT_COUNT            object
dtype: object

In [5]:
post_sec_df.count()

SCHOOL_YEAR             228739
AGENCY_TYPE             226669
CHARTER_IND             226669
CESA                    226669
COUNTY                  226669
DISTRICT_ID             228739
SCHOOL_CODE             124358
GRADE_GROUP             228739
DISTRICT_NAME           228739
SCHOOL_NAME             228739
INITIAL_ENROLLMENT      228739
INSTITUTION_LOCATION    228739
INSTITUTION_LEVEL       228739
INSTITUTION_TYPE        228739
GROUP_BY                228739
GROUP_BY_VALUE          228739
GROUP_COUNT             228739
STUDENT_COUNT           228739
dtype: int64

In [6]:
# Remove aggregate data with [Statewide] distinction
post_sec_df = post_sec_df[post_sec_df['DISTRICT_ID'] != 0]
post_sec_df.head(10)

,SCHOOL_YEAR,AGENCY_TYPE,CHARTER_IND,CESA,COUNTY,DISTRICT_ID,SCHOOL_CODE,GRADE_GROUP,DISTRICT_NAME,SCHOOL_NAME,INITIAL_ENROLLMENT,INSTITUTION_LOCATION,INSTITUTION_LEVEL,INSTITUTION_TYPE,GROUP_BY,GROUP_BY_VALUE,GROUP_COUNT,STUDENT_COUNT
576,2019-20,School District,No,10.0,Clark,7,NaN,[All],Abbotsford,[Districtwide],First Fall,In-State,2-Year,Public,All Students,All Students,21,4
577,2019-20,School District,No,10.0,Clark,7,NaN,[All],Abbotsford,[Districtwide],First Fall,In-State,4-Year,Public,All Students,All Students,21,12
578,2019-20,School District,No,10.0,Clark,7,NaN,[All],Abbotsford,[Districtwide],First Fall,In-State,Multiple,Public,All Students,All Students,21,1
579,2019-20,School District,No,10.0,Clark,7,NaN,[All],Abbotsford,[Districtwide],First Fall,Out-of-State,4-Year,Private,All Students,All Students,21,1
580,2019-20,School District,No,10.0,Clark,7,NaN,[All],Abbotsford,[Districtwide],Second Fall,In-State,4-Year,Private,All Students,All Students,21,1
581,2019-20,School District,No,10.0,Clark,7,NaN,[All],Abbotsford,[Districtwide],Second Fall,In-State,4-Year,Public,All Students,All Students,21,2
582,2019-20,School District,No,10.0,Clark,7,NaN,[All],Abbotsford,[Districtwide],First Fall,In-State,2-Year,Public,Disability Status,SwoD,21,4
583,2019-20,School District,No,10.0,Clark,7,NaN,[All],Abbotsford,[Districtwide],First Fall,In-State,4-Year,Public,Disability Status,SwoD,21,12
584,2019-20,School District,No,10.0,Clark,7,NaN,[All],Abbotsford,[Districtwide],First Fall,In-State,Multiple,Public,Disability Status,SwoD,21,1
585,2019-20,School District,No,10.0,Clark,7,NaN,[All],Abbotsford,[Districtwide],First Fall,Out-of-State,4-Year,Private,Disability Status,SwoD,21,1


In [7]:
post_sec_df = post_sec_df[post_sec_df['STUDENT_COUNT'] != '*']
post_sec_df.count()

SCHOOL_YEAR             212753
AGENCY_TYPE             212753
CHARTER_IND             212753
CESA                    212753
COUNTY                  212753
DISTRICT_ID             212753
SCHOOL_CODE             115830
GRADE_GROUP             212753
DISTRICT_NAME           212753
SCHOOL_NAME             212753
INITIAL_ENROLLMENT      212753
INSTITUTION_LOCATION    212753
INSTITUTION_LEVEL       212753
INSTITUTION_TYPE        212753
GROUP_BY                212753
GROUP_BY_VALUE          212753
GROUP_COUNT             212753
STUDENT_COUNT           212753
dtype: int64

In [8]:
# replace '*' with NaN
post_sec_df = post_sec_df.replace('*', np.NaN)


In [9]:
# Remove aggregate data with [Districtwide] distinction
post_sec_df = post_sec_df[post_sec_df['SCHOOL_NAME'] != '[Districtwide]']

In [10]:
post_sec_df.count()

SCHOOL_YEAR             115830
AGENCY_TYPE             115830
CHARTER_IND             115830
CESA                    115830
COUNTY                  115830
DISTRICT_ID             115830
SCHOOL_CODE             115830
GRADE_GROUP             115830
DISTRICT_NAME           115830
SCHOOL_NAME             115830
INITIAL_ENROLLMENT      115830
INSTITUTION_LOCATION    115830
INSTITUTION_LEVEL       115830
INSTITUTION_TYPE        115830
GROUP_BY                115830
GROUP_BY_VALUE          115830
GROUP_COUNT             115830
STUDENT_COUNT           115830
dtype: int64

In [14]:
# Typecast numeric data
post_sec_df['CESA'] = post_sec_df['CESA'].astype(int)
post_sec_df['SCHOOL_CODE'] = post_sec_df['SCHOOL_CODE'].astype(int)
post_sec_df['GROUP_COUNT'] = post_sec_df['GROUP_COUNT'].astype(int)
post_sec_df['STUDENT_COUNT'] = post_sec_df['STUDENT_COUNT'].astype(int)

In [15]:
# Change Charter School Yes/No to a boolean
post_sec_df['CHARTER_IND'].replace(('Yes', 'No'), (True, False), inplace=True)

In [16]:
post_sec_df.dtypes

SCHOOL_YEAR             object
AGENCY_TYPE             object
CHARTER_IND               bool
CESA                     int64
COUNTY                  object
DISTRICT_ID              int64
SCHOOL_CODE              int64
GRADE_GROUP             object
DISTRICT_NAME           object
SCHOOL_NAME             object
INITIAL_ENROLLMENT      object
INSTITUTION_LOCATION    object
INSTITUTION_LEVEL       object
INSTITUTION_TYPE        object
GROUP_BY                object
GROUP_BY_VALUE          object
GROUP_COUNT              int64
STUDENT_COUNT            int64
dtype: object

In [20]:
post_sec_df.to_csv('post_sec_clean.csv')